In [1]:
using Distributions, Plots

In [221]:
candidate_n = 10 # Number of validator candidates
validator_n = 5 # Number of validator slots
nominator_n = 10; # Number of nominator candidates

In [224]:
# Nominator preferences, determined by their stake, preferred return and validators
nomination_preference(ordered_candidates::Vector, subset::Vector) = rand(subset, rand(1:length(ordered_candidates)))

nomination_preference (generic function with 2 methods)

In [230]:
candidate_stake(candidate_n::Int) = sort(rand(Pareto(1, 1), candidate_n)) # Validators represented only by their stake, always ordered
nominator_stake(nominator_n::Int) = sort(rand(Pareto(1, 1), nominator_n)) # Nominator stakes
nominator_interest(nominator_n::Int) = rand(Beta(10, 100), nominator_n) # Desired nominator interests

nominator_interest (generic function with 2 methods)

In [231]:
# Total stake
total_stake(validators::Vector, nominations) = validators + sum(nominations, 2)[:]
inflation(stakes::Vector, rates::Vector) = (rates' * stakes)[1] / sum(stakes)
inflation(validators::Vector, nominations, rates::Vector) = inflation(total_stake(validators, nominations), rates)
# Threshold
threshold(validator_n::Int, candidates::Vector, nominations) = minimum(sort(sum(nominations, 2)[:]+candidates)[end-validator_n+1:end])

threshold (generic function with 2 methods)

In [238]:
function sample(candidate_n::Int, nominator_n::Int, validator_n::Int, strat::Function, sample_n::Int)
    infs = zeros(sample_n)
    ths = zeros(sample_n)
    for s in 1:sample_n
        nominators = nominator_stake(nominator_n)
        candidates = candidate_stake(candidate_n)
        interests = nominator_interest(nominator_n);
        rates, nominations = strat(candidate_n, candidates, nominator_n, nominators, interests)
        stakes = total_stake(candidates, nominations)
        infs[s] = inflation(stakes, rates)
        ths[s] = threshold(validator_n, candidates, nominations)/sum(stakes)
    end
    (infs, ths)
end

sample (generic function with 1 method)

In [239]:
function brute_strat(candidate_n::Int, candidates::Vector, nominator_n::Int, nominators::Vector, interests::Vector)
    rates = rand(Beta(10, 100), candidate_n)
    nominations = zeros(candidate_n, nominator_n)
    subset = rand(1:candidate_n, 5)
    for n in 1:nominator_n
        preference = nomination_preference(candidates, subset)
        c = length(preference)
        for p in preference
            if rates[p] >= interests[n]
                nominations[p, n] += nominators[n]/c
            end
        end
    end
    (rates, nominations)
end

brute_strat (generic function with 1 method)

In [240]:
infs, ths = sample(candidate_n, nominator_n, validator_n, brute_strat, 1000)

([0.0715082, 0.133519, 0.0899038, 0.111753, 0.0975577, 0.0931087, 0.0817353, 0.0818681, 0.108878, 0.0973602  …  0.0900733, 0.111527, 0.0932247, 0.0908907, 0.0926027, 0.0961822, 0.0824008, 0.101685, 0.0806159, 0.101641], [0.0576866, 0.0488898, 0.0394256, 0.0587774, 0.069917, 0.0601946, 0.0394191, 0.0693863, 0.0437943, 0.0653069  …  0.0663371, 0.0719057, 0.0371177, 0.046679, 0.0469944, 0.104592, 0.0457152, 0.0518623, 0.0380676, 0.0926505])

In [241]:
nominations

10×10 Array{Float64,2}:
 0.504046  0.82858  1.22      1.58988  2.18573  …  0.0  1.0183   0.0  0.0    
 0.0       0.0      0.0       0.0      0.0         0.0  0.0      0.0  0.0    
 0.0       0.0      0.0       0.0      0.0         0.0  0.0      0.0  0.0    
 0.33603   0.0      0.0       0.0      0.0         0.0  0.0      0.0  0.0    
 0.0       0.0      0.0       0.0      0.0         0.0  0.0      0.0  0.0    
 0.0       0.0      0.0       0.0      0.0      …  0.0  0.0      0.0  0.0    
 0.0       0.0      0.0       0.0      0.0         0.0  0.0      0.0  0.0    
 0.33603   0.0      0.304999  0.0      0.0         0.0  0.50915  0.0  0.0    
 0.0       0.0      0.0       0.0      0.0         0.0  0.0      0.0  0.0    
 0.0       0.0      0.304999  0.0      0.0         0.0  0.50915  0.0  1.51191

In [242]:
scatter(infs, ths)

In [243]:
plot(rates)